In [1]:
# ! pip install xgboost


In [148]:
import os
import sys
sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time




from sklearn import metrics, preprocessing
# from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score, ndcg_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# import config

import data_process.neg_sample as ng_sample
from data_process.utils import mix_merge
from data_process.data_split import data_split_user
from evaluate_entity import CustomHR, CustomNDCG, CustomRoc, CustomRoctop, CustomRecall_top, CustomPrecision_top
from model_entity import EntityCat
from data_utils import CatData
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL


# import argparse
torch.manual_seed(0)

In [52]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [53]:
device

device(type='cpu')

In [54]:
 device.type=='cpu'

True

In [55]:
if device =='cpu':
    BATCH_SIZE = 20
else:
    BATCH_SIZE = 2**20
EPOCHS  = 20
TOP_K = 10
NEG_TEST = 100
NEG_TRAIN = 4

In [74]:
df_train1  = ng_sample.read_feather("../../data/jobs/leave_one_train.csv")
df_train2 = pd.read_csv("../../data/jobs/leave_one_train_neg.csv")
df_test_ori = pd.read_csv("../../data/jobs/test_pos_neg.csv")
df_all_features = pd.read_csv('../../data/jobs/merged_sub_clean.csv')
df_train1 = df_train1.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
df_train2 = df_train2.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [75]:
df_train1[DEFAULT_RATING_COL] = 1

In [76]:
def concat_index(df1, df2, NEG_TRAIN):
    df2.index = df2.index//NEG_TRAIN
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [77]:
df_train_all = concat_index(df_train1, df_train2, NEG_TRAIN)

In [78]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

### Choose the features and process data for the training

In [79]:
user_features = ['WindowID_user', 'Split', 'City', 'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate', 'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed', 'ManagedOthers', 'ManagedHowMany']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = ['WindowID_job', 'City_job', 'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate']
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL]

In [80]:
df_mix_merge = mix_merge(df_all, df_all_features, user_features_extend, item_features_extend)

In [81]:
def _cat_encode(df_data, list_f, encoder):
    for f in list_f:
        df_data[f] = encoder.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [82]:
def _embedding_dimension(df_all_encode, features_to_train, max_dim=50):

    embedding_size = []
    features_to_em = [i for i in features_to_train if i !=DEFAULT_RATING_COL]
    for c in features_to_em:
        num_unique_values = int(df_all_encode[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_values/2), max_dim))
        embedding_size.append([num_unique_values, embed_dim])  
    return embedding_size

In [83]:
def encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=50):
    encoder = preprocessing.LabelEncoder()
#     features_to_code = df_mix_merge.columns
    df_all_encode = _cat_encode(df_mix_merge, features_to_code, encoder)
    df_train = df_all_encode[df_all.flag==1]
    df_test = df_all_encode[df_all.flag==0]
    df_train = df_train[features_to_train]
    df_test = df_test[features_to_train]
    embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
    return df_train, df_test, embedding_size

In [84]:
num_feature=[]
features_to_code = df_mix_merge.columns
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features +[DEFAULT_RATING_COL]
df_train,  df_test, embedding_size = encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=50)

print(f'The size of embedding layers:{embedding_size}')

The size of embedding layers:[[89946, 50], [139292, 50], [3, 2], [2, 1], [6296, 50], [55, 28], [1, 1], [12643, 50], [7, 4], [14075, 50], [606, 50], [20, 10], [64, 32], [3, 2], [2, 1], [251, 50], [4, 2], [5405, 50], [55, 28], [4, 2], [10465, 50], [139259, 50], [235, 50]]


In [85]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [86]:
df_test

,userid,itemid,WindowID_user,Split,City,State,Country,Zip_user,DegreeType,Major,...,ManagedOthers,ManagedHowMany,WindowID_job,City_job,State_job,Country_job,Zip_job,StartDate,EndDate,rating
2736740,0,86875,0,1,4716,48,0,3129,2,10123,...,0,0,1,4071,49,3,8,2443,16,1
2736741,0,63735,0,1,4716,48,0,3129,2,10123,...,0,0,1,2597,36,3,1398,25482,61,0
2736742,0,67228,0,1,4716,48,0,3129,2,10123,...,0,0,2,2236,46,3,8200,50432,88,0
2736743,0,46363,0,1,4716,48,0,3129,2,10123,...,0,0,3,194,11,3,3504,120611,159,0
2736744,0,119351,0,1,4716,48,0,3129,2,10123,...,0,0,1,1744,15,3,6828,27061,65,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219926,89945,60390,1,1,2582,17,0,6856,4,4198,...,0,0,1,2061,7,3,481,60390,97,0
10219927,89945,133921,1,1,2582,17,0,6856,4,4198,...,0,0,3,4275,5,3,9656,115798,156,0
10219928,89945,99424,1,1,2582,17,0,6856,4,4198,...,0,0,1,876,15,3,8,39343,80,0
10219929,89945,33346,1,1,2582,17,0,6856,4,4198,...,0,0,2,2988,45,3,8,102504,139,0


In [87]:
df_train_split

,userid,itemid,WindowID_user,Split,City,State,Country,Zip_user,DegreeType,Major,...,ManagedOthers,ManagedHowMany,WindowID_job,City_job,State_job,Country_job,Zip_job,StartDate,EndDate,rating
0,0,38336,0,1,4716,48,0,3129,2,10123,...,0,0,1,4071,49,3,8,32089,66,1
1,0,63730,0,1,4716,48,0,3129,2,10123,...,0,0,1,44,36,3,1412,24702,62,0
2,0,67227,0,1,4716,48,0,3129,2,10123,...,0,0,2,1194,23,3,5862,41679,81,0
3,0,46359,0,1,4716,48,0,3129,2,10123,...,0,0,2,2236,46,3,8182,26675,65,0
4,0,119347,0,1,4716,48,0,3129,2,10123,...,0,0,2,1198,11,3,8,36361,76,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736735,89945,24783,1,1,2582,17,0,6856,4,4198,...,0,0,2,2291,16,3,5550,36629,76,1
2736736,89945,67227,1,1,2582,17,0,6856,4,4198,...,0,0,2,1194,23,3,5862,41679,81,0
2736737,89945,46359,1,1,2582,17,0,6856,4,4198,...,0,0,2,2236,46,3,8182,26675,65,0
2736738,89945,63730,1,1,2582,17,0,6856,4,4198,...,0,0,1,44,36,3,1412,24702,62,0


In [88]:
df_val_split

,userid,itemid,WindowID_user,Split,City,State,Country,Zip_user,DegreeType,Major,...,ManagedOthers,ManagedHowMany,WindowID_job,City_job,State_job,Country_job,Zip_job,StartDate,EndDate,rating
75,5,120369,2,1,1429,12,0,4020,1,8784,...,0,0,3,2618,11,3,3390,79540,113,1
76,5,18934,2,1,1429,12,0,4020,1,8784,...,0,0,1,3789,43,3,3319,11011,40,0
77,5,67397,2,1,1429,12,0,4020,1,8784,...,0,0,3,4681,40,3,8,99445,137,0
78,5,80624,2,1,1429,12,0,4020,1,8784,...,0,0,1,999,6,3,8,16080,48,0
79,5,92911,2,1,1429,12,0,4020,1,8784,...,0,0,2,194,11,3,8,114569,152,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736695,89940,132088,2,1,2835,46,0,10361,4,8677,...,1,10,3,214,46,3,8490,121544,166,1
2736696,89940,52091,2,1,2835,46,0,10361,4,8677,...,1,10,3,3779,40,3,2025,135425,195,0
2736697,89940,46358,2,1,2835,46,0,10361,4,8677,...,1,10,1,3783,4,3,8836,25433,62,0
2736698,89940,67228,2,1,2835,46,0,10361,4,8677,...,1,10,2,2236,46,3,8200,50432,88,0


In [89]:
X_train_split = df_train_split.iloc[:,:-1]
y_train_split = df_train_split.iloc[:,-1]

X_val_split = df_val_split.iloc[:,:-1]
y_val_split = df_val_split.iloc[:,-1]


X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]



In [90]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'



In [91]:
train_dmatrix = xgb.DMatrix(data=X_train_split,label=y_train_split)
val_dmatrix = xgb.DMatrix(data=X_val_split,label=y_val_split)
test_dmatrix = xgb.DMatrix(data=X_test)#,label=y_test)

In [92]:
evallist = [(val_dmatrix, 'eval'), (train_dmatrix, 'train')]

In [100]:
num_round = 400
bst = xgb.train(param, train_dmatrix, num_round, evallist, early_stopping_rounds=10)
# train(..., evals=evals, early_stopping_rounds=10)
# bst.save_model('test0001.model')

/Users/che/miniforge3/envs/torch/lib/python3.9/site-packages/xgboost/core.py:525: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


[0]	eval-auc:0.56509	train-auc:0.56509
[1]	eval-auc:0.62976	train-auc:0.62931
[2]	eval-auc:0.65019	train-auc:0.64920
[3]	eval-auc:0.70628	train-auc:0.70659
[4]	eval-auc:0.71468	train-auc:0.71529
[5]	eval-auc:0.80280	train-auc:0.80390
[6]	eval-auc:0.80595	train-auc:0.80670
[7]	eval-auc:0.81223	train-auc:0.81256
[8]	eval-auc:0.81431	train-auc:0.81513
[9]	eval-auc:0.82025	train-auc:0.82151
[10]	eval-auc:0.82205	train-auc:0.82294
[11]	eval-auc:0.84627	train-auc:0.84730
[12]	eval-auc:0.84978	train-auc:0.85118
[13]	eval-auc:0.85393	train-auc:0.85482
[14]	eval-auc:0.85762	train-auc:0.85868
[15]	eval-auc:0.86250	train-auc:0.86348
[16]	eval-auc:0.86615	train-auc:0.86698
[17]	eval-auc:0.87018	train-auc:0.87089
[18]	eval-auc:0.87353	train-auc:0.87431
[19]	eval-auc:0.87738	train-auc:0.87812
[20]	eval-auc:0.87921	train-auc:0.88003
[21]	eval-auc:0.91139	train-auc:0.91161
[22]	eval-auc:0.91245	train-auc:0.91269
[23]	eval-auc:0.91282	train-auc:0.91316
[24]	eval-auc:0.91341	train-auc:0.91373
[25]	eval-

[203]	eval-auc:0.99097	train-auc:0.99151
[204]	eval-auc:0.99105	train-auc:0.99157
[205]	eval-auc:0.99105	train-auc:0.99158
[206]	eval-auc:0.99107	train-auc:0.99159
[207]	eval-auc:0.99107	train-auc:0.99160
[208]	eval-auc:0.99110	train-auc:0.99163
[209]	eval-auc:0.99116	train-auc:0.99169
[210]	eval-auc:0.99124	train-auc:0.99176
[211]	eval-auc:0.99124	train-auc:0.99177
[212]	eval-auc:0.99130	train-auc:0.99181
[213]	eval-auc:0.99131	train-auc:0.99183
[214]	eval-auc:0.99131	train-auc:0.99184
[215]	eval-auc:0.99125	train-auc:0.99180
[216]	eval-auc:0.99125	train-auc:0.99181
[217]	eval-auc:0.99129	train-auc:0.99182
[218]	eval-auc:0.99134	train-auc:0.99187
[219]	eval-auc:0.99138	train-auc:0.99191
[220]	eval-auc:0.99152	train-auc:0.99203
[221]	eval-auc:0.99164	train-auc:0.99211
[222]	eval-auc:0.99167	train-auc:0.99214
[223]	eval-auc:0.99159	train-auc:0.99209
[224]	eval-auc:0.99161	train-auc:0.99210
[225]	eval-auc:0.99163	train-auc:0.99213
[226]	eval-auc:0.99162	train-auc:0.99213
[227]	eval-auc:0

In [101]:
test_preds = bst.predict(test_dmatrix)


In [102]:
roc_auc_score(y_test, test_preds)

0.9923712265676514

In [ ]:
metric_names=['loss',"auc", 'hr', 'ndcg', 'roc_top', 'recall_top', 'precision_top'],

In [104]:
df_test['y_pred'] = test_preds

In [120]:
# device = 'cpu'
def hit(gt_item, pred_items):
    if gt_item in pred_items:
        return 1
    return 0


def ndcg(gt_item, pred_items):
    if gt_item in pred_items:
        index = pred_items.index(gt_item)
        return np.reciprocal(np.log2(index + 2))
    return 0


def roc(gt_item, pred_prob):
    try:
        return roc_auc_score(gt_item, pred_prob)
    except:
        return 0

def recall(gt, prob, th):
    y_pred = [1 if y >= th else 0 for y in prob]
    return recall_score(gt, y_pred, zero_division=0)

def precision(gt, prob, th):
    y_pred = [1 if y >= th else 0 for y in prob]
    return precision_score(gt, y_pred, zero_division=0)

In [108]:
df_test_result = df_test[['userid', 'rating','y_pred', 'itemid']]

In [117]:
df_test_result_sort = df_test_result.sort_values(['userid','y_pred'], ascending = False)

In [118]:
df_test_result_sort.head()

,userid,rating,y_pred,itemid
10219830,89945,1,0.987102,30752
10219906,89945,0,0.946819,109812
10219878,89945,0,0.668708,47142
10219847,89945,0,0.655196,89889
10219904,89945,0,0.623258,101971


In [130]:
precision(y_test, test_preds, 0.5)

0.21256247132958303

In [171]:
df_test_result_sort.iloc[:17,:][df_test_result_sort.iloc[:17,:].rating==1].itemid.tolist()[0]

30752

In [172]:
topk = 10
hr_ls = []
ndcg_ls = []
roc_ls = []
recall_ls = []
precision_ls = []

def eval_scores(df, topk,  thres = 0.5):
    df_topk = df.iloc[:topk]
    gt_item =  df[df.rating==1].itemid.tolist()[0]
    pred_items = df_topk.itemid.tolist()
#     print("gt_item, pred_items ", gt_item, pred_items)
    
    hr_user = hit(gt_item, pred_items)

    y_true = [df.rating.tolist()]
    y_score = [df.y_pred.tolist()]
    
    ndcg_user = ndcg_score(y_true, y_score, k=topk)
    
    y_gt = df_topk.rating.tolist()
    y_pred =  df_topk.y_pred.tolist()
    
    roc_user = roc(y_gt, y_pred)
    recall_user = recall(y_gt, y_pred, thres)
    precision_user =  precision(y_gt, y_pred, thres)
    
    hr_ls.append(hr_user)
    ndcg_ls.append(ndcg_user)
    roc_ls.append(roc_user)
    recall_ls.append(recall_user)
    precision_ls.append(precision_user)

for  thres in [0.5, 0.7, 0.9, 0.97]: 
    df_test_result_sort.groupby('userid').apply(lambda df:eval_scores(df, topk, thres) )
    avg_hr = np.mean(hr_ls)
#     print("The prob threshold is: ", thres)
    
#     print(avg_hr)
#     print("-"*30) 
#     break
    avg_ndcg = np.mean(ndcg_ls)
    
    avg_roc = np.mean(roc_ls )
    avg_recall = np.mean(recall_ls) 
    avg_precision = np.mean(precision_ls )
    print("The prob threshold is: ", thres)
    print("avg_hr: ", avg_hr, "\navg_ndcg: ",avg_ndcg, "\navg_roc: ", avg_roc,  "\navg_recall: ", avg_recall, "\navg_precision: ", avg_precision)
    print("-"*30)

The prob threshold is:  0.5
avg_hr:  0.9824135185110202 
avg_ndcg:  0.8794684331620457 
avg_roc:  0.9281131761392523 
avg_recall:  0.9498454603123186 
avg_precision:  0.2912561568967206
------------------------------
The prob threshold is:  0.7
avg_hr:  0.9824135185110202 
avg_ndcg:  0.8794684331620459 
avg_roc:  0.9281131761392523 
avg_recall:  0.9314154215761699 
avg_precision:  0.3662185262096183
------------------------------
The prob threshold is:  0.9
avg_hr:  0.9824135185110202 
avg_ndcg:  0.8794684331620459 
avg_roc:  0.9281131761392525 
avg_recall:  0.8738713204032879 
avg_precision:  0.4434103343100388
------------------------------
The prob threshold is:  0.97
avg_hr:  0.9824135185110202 
avg_ndcg:  0.879468433162046 
avg_roc:  0.9281131761392523 
avg_recall:  0.7616478384689098 
avg_precision:  0.4331653368990901
------------------------------


In [160]:
y_gt = [df_test_result_sort.rating.tolist()]
y_pred =  [df_test_result_sort.y_pred.tolist()]
ndcg_score(y_gt, y_pred, k=4)

0.9999999999999998

In [159]:
>>> import numpy as np
>>> from sklearn.metrics import ndcg_score
>>> # we have groud-truth relevance of some answers to a query:
>>> true_relevance = np.asarray([[10, 0, 0, 1, 5]])
>>> # we predict some scores (relevance) for the answers
>>> scores = np.asarray([[.1, .2, .3, 4, 70]])
>>> ndcg_score(true_relevance, scores, k=4)

0.4123818817534531

In [146]:


#Precision threshold 
avg_hr:  0.9824135185110202 
avg_roc:  0.9281131761392523 
avg_recall:  0.9498454603123186 
avg_precision:  0.59





avg_hr:  0.9824135185110202 
avg_roc:  0.9281131761392523 
avg_recall:  0.9498454603123186 
avg_precision:  0.4024303446662438